In [1]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

## Carga de información

In [2]:
laliga2122 = pd.read_excel('../Semana 3/datos_liga/LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('../Semana 3/datos_liga/LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('../Semana 3/datos_liga/LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('../Semana 3/datos_liga/LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('../Semana 3/datos_liga/LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('../Semana 3/datos_liga/LaLiga16 17.xlsx', )
seg2122 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 21 22.xlsx', )
seg2021 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 20 21.xlsx', )
seg1920 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 19 20.xlsx', )
seg1819 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 18 19.xlsx', )
seg1718 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 17 18.xlsx', )
seg1617 =  pd.read_excel('../Semana 3/datos_liga/LaLiga123 16 17.xlsx', )

## Ejemplo 1
Se crea la database de primera división

In [3]:
laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all'))
laliga2 = laliga.loc[:,['Wk','Date','Local','xG','Score','xG.1','Visitante', 'Attendance']]

In [4]:
def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])

def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

laliga2['Attendance']= laliga2['Attendance']*1000
laliga2['Goles Local'] = laliga2['Score'].apply(goleslocal)
laliga2['Goles Visitante'] = laliga2['Score'].apply(golesvisita)

Vamos a estudiar la cantidad de goles que ha realizado el Real Madrid y el Barcelona, de local, durante las últimas 6 temporadas.

In [5]:
laliga2[laliga2['Local']=='Real Madrid']['Goles Local'].sum() #calculamos el total (suma) de goles de local del Real Madrid (con la condición que el equipo local sea el Real Madrid)

251

In [6]:
laliga2[laliga2['Local']=='Barcelona']['Goles Local'].sum()

301

In [7]:
laliga2[laliga2['Local']=='Real Madrid']['Goles Local'].mean() #misma cosa que para los goles totales, pero calculamos el promedio (mean)

2.2017543859649122

In [8]:
laliga2[laliga2['Local']=='Barcelona']['Goles Local'].mean()

2.6403508771929824

In [9]:
laliga2[laliga2['Local']=='Real Madrid']['Goles Local'].describe() #con el comando "describe" obtenemos los siguientes estadísticos descriptivos 

count    114.000000
mean       2.201754
std        1.552502
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        7.000000
Name: Goles Local, dtype: float64

In [10]:
laliga2[laliga2['Local']=='Barcelona']['Goles Local'].describe()

count    114.000000
mean       2.640351
std        1.730145
min        0.000000
25%        1.000000
50%        2.000000
75%        4.000000
max        8.000000
Name: Goles Local, dtype: float64

Ahora, estudiaremos la asistencia promedio por temporada/torneo.

In [11]:
#construimos una función que nos retorne una de las 6 temporadas si le indicamos año y mes del partido
def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<8)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

    
laliga2['Año'] = laliga2['Date'].dt.year #sacamos el año del partido y lo agregamos a la nueva columna "Año"
laliga2['Mes'] = laliga2['Date'].dt.month #sacamos el mes del partido y lo agregamos a la nueva columna "Mes"
laliga2['Temporada'] = laliga2.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1) #aplicamos la función lambda para aplicar la función torneo al mes y año de un partido


In [12]:
temporadas=['16/17','17/18','18/19','19/20','20/21','21/22'] #creamos una lista con todas las temporadas
asist = {} #creamos el diccionario "asist"
for temporada in temporadas: #recorremos todas las temporadas
    base = laliga2[laliga2['Temporada']==temporada] #creamos un Dataframe solo con información de la temporada (de primera división) en que estamos parados
    prom = base['Attendance'].mean() #calculamos el promedio de asistencia de la temporada en la que estamos parados (sobre el Dataframe base)
    total = base['Attendance'].sum() #calculamos la asistencia total de la temporada en la que estamos parados (sobre el Dataframe base)
    asist[temporada] = (total,prom) #guardamos el promedio y total obtenidos en el diccionario, con "key" nombre de la temporada

In [13]:
#transformamos el diccionario en Dataframe
asistencia = pd.DataFrame([[key, asist[key][0],asist[key][1]] for key in asist.keys()], 
                        columns=['Temporada', 'Asistencia Total','Asistencia promedio'])
asistencia

,Temporada,Asistencia Total,Asistencia promedio
0,16/17,10737706.0,28257.121053
1,17/18,10528159.0,27778.783641
2,18/19,10305100.0,27118.684211
3,19/20,7877083.0,29282.836431
4,20/21,12745.0,3186.250000
5,21/22,9308092.0,24494.978947


## Ejercicio  1
¿Cuál es el código para calcular la cantidad de partidos que jugó en primera división el Alavés?


In [14]:
#filtramos el Dataframe laliga buscando al Alavés, ya sea de local o de visitante. Contamos las fechas en las que aparece.
laliga2[(laliga2['Local']=='Alavés')|(laliga2['Visitante']=='Alavés')]['Date'].count()

228

## Ejercicio 2
¿Cómo determinamos ahora la asistencia por año calendario (de enero a diciembre de cada año?

In [15]:
años=[2016+i for i in range(7)] #construimos la lista de años, de 2016 a 2016+6=2022 (range(7) = 6)
asist2 = {} #creamos este diccionario 
for año in años: #recorremos los años en la lista
    base = laliga2[laliga2['Año']==año] #filtramos el Dataframe laliga solo para los registros del año en que estamos parados
    prom = base['Attendance'].mean()
    total = base['Attendance'].sum()
    asist2[año] = (total,prom) #guardamos el promedio y total obtenidos en el diccionario, con "key" nombre del año

In [16]:
#transformamos en Dataframe el diccionario
asistencia2 = pd.DataFrame([[key, asist2[key][0],asist2[key][1]] for key in asist2.keys()], 
                        columns=['Año', 'Asistencia Total','Asistencia promedio'])
asistencia2

,Año,Asistencia Total,Asistencia promedio
0,2016,4545731.0,28589.503145
1,2017,10852300.0,27897.943445
2,2018,10573664.0,27825.431579
3,2019,10795892.0,27610.976982
4,2020,2680461.0,30117.539326
5,2021,4325846.0,23257.236559
6,2022,4994991.0,25227.227273
